We initialize Python imports and opens a DuckDB connection that every later cell reuses

In [1]:
import duckdb
import pandas as pd

from pathlib import Path

cwd = Path.cwd()

root = None
for p in [cwd] + list(cwd.parents):
    if (p / "db").exists():
        root = p
        break

if root is None:
    raise FileNotFoundError("Could not find a 'db' folder above the current working directory")

DB_PATH = root / "db" / "nflpa.duckdb"

print("Using DB_PATH", DB_PATH)

con = duckdb.connect(str(DB_PATH))

con.execute("PRAGMA threads=4")
con.execute("PRAGMA memory_limit='4GB'")

Using DB_PATH /Users/ramko/Desktop/2025-26-NFLPA-Data-Analytics-Case-Competition/db/nflpa.duckdb


Quick sanity check to confirm that the target table exists in the database and contains the necessary special teams metrics from the previous step to ensure the environment is ready for statistical analysis

In [2]:
required_tables = ["team_week_panel", "schedules"]
existing = set(con.execute("SHOW TABLES").df()["name"].tolist())
missing_tables = [t for t in required_tables if t not in existing]

print("Missing tables", missing_tables)
print("OK" if not missing_tables else "STOP, fix missing tables before continuing")

panel_cols_df = con.execute("DESCRIBE team_week_panel").df()
panel_cols = set(panel_cols_df["column_name"].tolist())

required_cols = [
    "season",
    "week",
    "game_id",
    "ST_Load_All_w",
    "ST_Load_ScoreLinked_w",
    "ST_Load_NonScore_w",
]

missing_cols = [c for c in required_cols if c not in panel_cols]

print("Missing required columns in team_week_panel", missing_cols)
print("OK" if not missing_cols else "STOP, Step 4 outputs missing from panel")

Missing tables []
OK
Missing required columns in team_week_panel []
OK


We detect the primary team join key by inspecting the panel schema, allowing the downstream processing logic to remain agnostic to naming conventions between different data versions

In [3]:
if "team_id" in panel_cols:
    PANEL_TEAM_COL = "team_id"
elif "team" in panel_cols:
    PANEL_TEAM_COL = "team"
else:
    raise ValueError("Could not find a team column in team_week_panel")

print("Using panel team column", PANEL_TEAM_COL)

Using panel team column team


We create a base view that identifies regular season game weeks to ensure that season-level team statistics are not skewed by bye weeks or post-season data

In [4]:
con.execute(f"""
CREATE OR REPLACE TEMP VIEW panel_step5_base AS
SELECT
  p.*,
  s.game_type AS sched_game_type,
  CASE WHEN p.game_id IS NOT NULL AND s.game_type = 'REG' THEN 1 ELSE 0 END AS is_reg_game_week
FROM team_week_panel p
LEFT JOIN schedules s
  ON p.game_id = s.game_id
""")

Quick sanity check to confirm that the schedule merge didn't accidentally multiply our data rows and also that every game week is correctly labeled as a regular or post-season game

In [5]:
con.execute(f"""
SELECT
  (SELECT COUNT(*) FROM team_week_panel) AS panel_rows,
  (SELECT COUNT(*) FROM panel_step5_base) AS base_rows,
  (SELECT COUNT(DISTINCT season || '-' || week || '-' || {PANEL_TEAM_COL}) FROM team_week_panel) AS panel_distinct_team_weeks,
  (SELECT COUNT(DISTINCT season || '-' || week || '-' || {PANEL_TEAM_COL}) FROM panel_step5_base) AS base_distinct_team_weeks,
  SUM(CASE WHEN game_id IS NOT NULL AND sched_game_type IS NULL THEN 1 ELSE 0 END) AS games_missing_schedule_type
FROM panel_step5_base
""").df()

,panel_rows,base_rows,panel_distinct_team_weeks,base_distinct_team_weeks,games_missing_schedule_type
0,6782,6782,6782,6782,0.0


We compute the mean and standard deviation for every workload category at the team-season level, using only active regular season weeks, and persist the results in a reference table for future normalization

In [6]:
con.execute(f"""
CREATE OR REPLACE TABLE team_season_st_stats AS
SELECT
  season,
  {PANEL_TEAM_COL} AS team,

  AVG(ST_Load_All_w) AS mean_ST_All,
  STDDEV_SAMP(ST_Load_All_w) AS sd_ST_All,

  AVG(ST_Load_ScoreLinked_w) AS mean_ST_ScoreLinked,
  STDDEV_SAMP(ST_Load_ScoreLinked_w) AS sd_ST_ScoreLinked,

  AVG(ST_Load_NonScore_w) AS mean_ST_NonScore,
  STDDEV_SAMP(ST_Load_NonScore_w) AS sd_ST_NonScore,

  COUNT(*) AS n_game_weeks
FROM panel_step5_base
WHERE is_reg_game_week = 1
GROUP BY season, {PANEL_TEAM_COL}
""")

Quick sanity check to confirm that each team-season has a unique entry in the summary table, ensuring the game counts align with a typical season length and that the standard deviation is successfully calculated

In [7]:
con.execute("""
SELECT
  COUNT(*) AS rows,
  COUNT(DISTINCT season || '-' || team) AS distinct_team_seasons,
  MIN(n_game_weeks) AS min_game_weeks,
  MAX(n_game_weeks) AS max_game_weeks,
  SUM(CASE WHEN sd_ST_All IS NULL OR sd_ST_All = 0 THEN 1 ELSE 0 END) AS zero_or_null_sd_all,
  SUM(CASE WHEN sd_ST_NonScore IS NULL OR sd_ST_NonScore = 0 THEN 1 ELSE 0 END) AS zero_or_null_sd_nonscore
FROM team_season_st_stats
""").df()

,rows,distinct_team_seasons,min_game_weeks,max_game_weeks,zero_or_null_sd_all,zero_or_null_sd_nonscore
0,416,416,16,17,17.0,17.0


Quick sanity check to confirm that we can catch cases where the standard deviation is zero or missing, ensuring we don't cause errors when dividing to calculate z-scores

In [8]:
con.execute("""
SELECT
  season,
  team,
  n_game_weeks,
  mean_ST_All,
  sd_ST_All,
  mean_ST_NonScore,
  sd_ST_NonScore
FROM team_season_st_stats
WHERE sd_ST_All IS NULL OR sd_ST_All = 0 OR sd_ST_NonScore IS NULL OR sd_ST_NonScore = 0
ORDER BY season, team
""").df()

,season,team,n_game_weeks,mean_ST_All,sd_ST_All,mean_ST_NonScore,sd_ST_NonScore
0,2012,OAK,16,0.0,0.0,0.0,0.0
1,2012,SD,16,0.0,0.0,0.0,0.0
2,2012,STL,16,0.0,0.0,0.0,0.0
3,2013,OAK,16,0.0,0.0,0.0,0.0
4,2013,SD,16,0.0,0.0,0.0,0.0
5,2013,STL,16,0.0,0.0,0.0,0.0
6,2014,OAK,16,0.0,0.0,0.0,0.0
7,2014,SD,16,0.0,0.0,0.0,0.0
8,2014,STL,16,0.0,0.0,0.0,0.0
9,2015,OAK,16,0.0,0.0,0.0,0.0


We join the team season statistics back to the individual game-week rows and calculate the z-scores, defaulting to zero if the standard deviation is zero or if the week isn't a regular season game to keep the data clean

In [9]:
con.execute(f"""
CREATE OR REPLACE TEMP VIEW panel_step5_z AS
SELECT
  p.*,

  ts.mean_ST_All,
  ts.sd_ST_All,
  ts.mean_ST_ScoreLinked,
  ts.sd_ST_ScoreLinked,
  ts.mean_ST_NonScore,
  ts.sd_ST_NonScore,
  ts.n_game_weeks,

  CASE
    WHEN ts.mean_ST_All IS NOT NULL AND ts.mean_ST_All > 0 THEN 1 ELSE 0
  END AS has_st_data_team_season,

  CASE
    WHEN p.is_reg_game_week = 1
     AND ts.mean_ST_All IS NOT NULL AND ts.mean_ST_All > 0
     AND ts.sd_ST_All IS NOT NULL AND ts.sd_ST_All > 0
      THEN (p.ST_Load_All_w - ts.mean_ST_All) / ts.sd_ST_All
    ELSE 0
  END AS Z_ST_All_w,

  CASE
    WHEN p.is_reg_game_week = 1
     AND ts.mean_ST_All IS NOT NULL AND ts.mean_ST_All > 0
     AND ts.sd_ST_ScoreLinked IS NOT NULL AND ts.sd_ST_ScoreLinked > 0
      THEN (p.ST_Load_ScoreLinked_w - ts.mean_ST_ScoreLinked) / ts.sd_ST_ScoreLinked
    ELSE 0
  END AS Z_ST_ScoreLinked_w,

  CASE
    WHEN p.is_reg_game_week = 1
     AND ts.mean_ST_All IS NOT NULL AND ts.mean_ST_All > 0
     AND ts.sd_ST_NonScore IS NOT NULL AND ts.sd_ST_NonScore > 0
      THEN (p.ST_Load_NonScore_w - ts.mean_ST_NonScore) / ts.sd_ST_NonScore
    ELSE 0
  END AS Z_ST_NonScore_w

FROM panel_step5_base p
LEFT JOIN team_season_st_stats ts
  ON p.season = ts.season
 AND p.{PANEL_TEAM_COL} = ts.team
""")

We compute a rolling version of the NonScore z-score and shock flags that only looks at games already played and also keeps the model safe from "predicting" the past using future information

In [10]:
con.execute(f"""
CREATE OR REPLACE TEMP VIEW panel_step5_rolling AS
SELECT
  *,
  AVG(ST_Load_NonScore_w) OVER (
    PARTITION BY season, {PANEL_TEAM_COL}
    ORDER BY week
    ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
  ) AS roll_mean_ST_NonScore,
  STDDEV_SAMP(ST_Load_NonScore_w) OVER (
    PARTITION BY season, {PANEL_TEAM_COL}
    ORDER BY week
    ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
  ) AS roll_sd_ST_NonScore
FROM panel_step5_base
WHERE is_reg_game_week = 1
""")

con.execute("""
CREATE OR REPLACE TEMP VIEW panel_step5_rolling_z AS
SELECT
  *,
  CASE
    WHEN roll_sd_ST_NonScore IS NOT NULL AND roll_sd_ST_NonScore > 0
      THEN (ST_Load_NonScore_w - roll_mean_ST_NonScore) / roll_sd_ST_NonScore
    ELSE 0
  END AS Z_ST_NonScore_Roll_w,
  CASE
    WHEN roll_sd_ST_NonScore IS NOT NULL AND roll_sd_ST_NonScore > 0
     AND (ST_Load_NonScore_w - roll_mean_ST_NonScore) / roll_sd_ST_NonScore >= 1
      THEN 1 ELSE 0
  END AS ST_Shock_NonScore_Roll_w
FROM panel_step5_rolling
""")

Quick sanity check to confirm that every regular season game has a valid z-score and that the values average out to approximately zero for each team's season, verifying the normalization was successful

In [11]:
con.execute("""
SELECT
  SUM(CASE WHEN is_reg_game_week = 1 AND mean_ST_All IS NULL THEN 1 ELSE 0 END) AS reg_weeks_missing_stats,
  SUM(CASE WHEN is_reg_game_week = 1 AND Z_ST_All_w IS NULL THEN 1 ELSE 0 END) AS reg_weeks_missing_z_all,
  SUM(CASE WHEN is_reg_game_week = 1 AND Z_ST_NonScore_w IS NULL THEN 1 ELSE 0 END) AS reg_weeks_missing_z_nonscore
FROM panel_step5_z
""").df()

,reg_weeks_missing_stats,reg_weeks_missing_z_all,reg_weeks_missing_z_nonscore
0,0.0,0.0,0.0


Quick sanity check to confirm that every regular season game has a valid z-score and that the values average out to approximately zero for each team's season, verifying the normalization was successful

In [12]:
con.execute(f"""
SELECT
  season,
  {PANEL_TEAM_COL} AS team,
  AVG(Z_ST_All_w) AS avg_z_all,
  AVG(Z_ST_ScoreLinked_w) AS avg_z_scorelinked,
  AVG(Z_ST_NonScore_w) AS avg_z_nonscore,
  COUNT(*) AS reg_rows
FROM panel_step5_z
WHERE is_reg_game_week = 1
GROUP BY season, {PANEL_TEAM_COL}
ORDER BY ABS(AVG(Z_ST_NonScore_w)) DESC
LIMIT 25
""").df()

,season,team,avg_z_all,avg_z_scorelinked,avg_z_nonscore,reg_rows
0,2024,PIT,6.138880e-16,-1.828603e-16,3.657205e-16,17
1,2023,CLE,5.240906e-16,1.828603e-16,3.461284e-16,17
2,2024,IND,-5.714383e-16,1.485740e-16,3.395976e-16,17
3,2022,ATL,6.465416e-16,-4.898043e-17,3.395976e-16,17
4,2021,CAR,-5.812344e-16,-6.530724e-17,3.273525e-16,17
5,2023,KC,-7.902176e-16,1.175530e-16,3.265362e-16,17
6,2023,NYJ,-2.742904e-16,8.489941e-17,3.265362e-16,17
7,2022,LV,-5.224579e-16,1.240837e-16,3.232708e-16,17
8,2022,ARI,5.093964e-16,-9.143013e-17,-3.134747e-16,17
9,2024,SEA,4.179663e-16,3.265362e-16,3.004133e-16,17


Quick sanity check to confirm that per week z scores have a reasonable spread and include both positive and negative values

In [13]:
con.execute(f"""
SELECT
  season,
  week,
  {PANEL_TEAM_COL} AS team,
  ST_Load_NonScore_w,
  mean_ST_NonScore,
  sd_ST_NonScore,
  Z_ST_NonScore_w
FROM panel_step5_z
WHERE is_reg_game_week = 1
ORDER BY ABS(Z_ST_NonScore_w) DESC
LIMIT 50
""").df()

,season,week,team,ST_Load_NonScore_w,mean_ST_NonScore,sd_ST_NonScore,Z_ST_NonScore_w
0,2019,10,LA,22.0,13.812500,2.638655,3.102907
1,2023,8,NYG,27.0,14.941176,3.928591,3.069504
2,2022,18,DAL,23.0,12.882353,3.333211,3.035406
3,2015,7,NO,24.0,14.187500,3.290770,2.981825
4,2023,14,JAX,22.0,12.647059,3.161115,2.958748
5,2019,6,TB,21.0,13.625000,2.552776,2.889012
6,2023,15,CHI,24.0,12.470588,4.032953,2.858802
7,2023,1,CIN,23.0,13.647059,3.277598,2.853596
8,2018,5,BAL,20.0,13.500000,2.280351,2.850439
9,2019,1,ARI,23.0,13.687500,3.280625,2.838636


Quick sanity check to confirm that z scores are not all near zero and have plausible min and max

In [14]:
con.execute("""
SELECT
  MIN(Z_ST_All_w) AS min_z_all,
  MAX(Z_ST_All_w) AS max_z_all,
  AVG(Z_ST_All_w) AS avg_z_all,

  MIN(Z_ST_ScoreLinked_w) AS min_z_scorelinked,
  MAX(Z_ST_ScoreLinked_w) AS max_z_scorelinked,
  AVG(Z_ST_ScoreLinked_w) AS avg_z_scorelinked,

  MIN(Z_ST_NonScore_w) AS min_z_nonscore,
  MAX(Z_ST_NonScore_w) AS max_z_nonscore,
  AVG(Z_ST_NonScore_w) AS avg_z_nonscore
FROM panel_step5_z
WHERE is_reg_game_week = 1
""").df()

,min_z_all,max_z_all,avg_z_all,min_z_scorelinked,max_z_scorelinked,avg_z_scorelinked,min_z_nonscore,max_z_nonscore,avg_z_nonscore
0,-2.875046,3.195742,-9.822085e-20,-2.915405,3.161037,-8.021370e-19,-2.728081,3.102907,-1.067333e-17


We create binary shock indicators by flagging any week where the z-score exceeds 1.0, ensuring that non-regular season weeks are explicitly set to zero to avoid misclassifying byes or off-weeks as high-workload events

In [15]:
con.execute("""
CREATE OR REPLACE TEMP VIEW panel_step5_shocks AS
SELECT
  p.*,

  CASE
    WHEN is_reg_game_week = 1 AND has_st_data_team_season = 1 AND Z_ST_All_w >= 1 THEN 1 ELSE 0
  END AS ST_Shock_All_w,

  CASE
    WHEN is_reg_game_week = 1 AND has_st_data_team_season = 1 AND Z_ST_ScoreLinked_w >= 1 THEN 1 ELSE 0
  END AS ST_Shock_ScoreLinked_w,

  CASE
    WHEN is_reg_game_week = 1 AND has_st_data_team_season = 1 AND Z_ST_NonScore_w >= 1 THEN 1 ELSE 0
  END AS ST_Shock_NonScore_w

FROM panel_step5_z p
""")

Quick sanity check to confirm that the incidence rate of the binary shock features aligns with the expected frequency of a+1σ threshold and that the is_regular_season filter is effectively preventing false positives in non-active weeks

In [16]:
con.execute("""
SELECT
  AVG(ST_Shock_All_w * 1.0) AS shock_rate_all,
  AVG(ST_Shock_ScoreLinked_w * 1.0) AS shock_rate_scorelinked,
  AVG(ST_Shock_NonScore_w * 1.0) AS shock_rate_nonscore,
  SUM(CASE WHEN is_reg_game_week = 0 AND (ST_Shock_All_w = 1 OR ST_Shock_ScoreLinked_w = 1 OR ST_Shock_NonScore_w = 1) THEN 1 ELSE 0 END) AS shocks_on_non_reg_rows
FROM panel_step5_shocks
""").df()

,shock_rate_all,shock_rate_scorelinked,shock_rate_nonscore,shocks_on_non_reg_rows
0,0.154232,0.155411,0.154674,0.0


We add standardized alias columns for the primary shock and z-score variables, streamlining the downstream modeling pipeline by pointing the algorithm to the NonScore metrics as the default inputs

In [17]:
con.execute("""
CREATE OR REPLACE TEMP VIEW panel_step5_final AS
SELECT
  p.*,
  ST_Shock_NonScore_w AS ST_Shock_Primary_w,
  Z_ST_NonScore_w AS Z_ST_Primary_w
FROM panel_step5_shocks p
""")

We persist the enriched feature set to the database by overwriting the existing panel, centralizing the newly computed z-scores and shock indicators to maintain a unified data state for the rest of the workflow

In [18]:
con.execute("""
CREATE OR REPLACE TABLE team_week_panel AS
SELECT * FROM panel_step5_final
""")

Quick sanity check to confirm that we cleaned up any duplicate columns that might have appeared during the joining process and also that each shock or z-score field only appears once in our final table

In [19]:
helper_bases = ["has_st_data_team_season", "is_reg_game_week", "sched_game_type"]

cols = con.execute("""
SELECT column_name
FROM duckdb_columns
WHERE table_name = 'team_week_panel'
""").df()["column_name"].tolist()

dupes = [c for c in cols if any(c.startswith(base + "_") for base in helper_bases)]

print("Duplicate helper columns to drop", dupes)

if dupes:
    exclude_list = ",\n    ".join(dupes)
    con.execute(f"""
    CREATE OR REPLACE TABLE team_week_panel AS
    SELECT
      * EXCLUDE (
        {exclude_list}
      )
    FROM team_week_panel
    """)
else:
    print("No duplicate helper columns found")

Duplicate helper columns to drop []
No duplicate helper columns found


Quick sanity check to confirm that every active game week shows at least some special teams activity for teams that are active and also ensuring we haven't accidentally dropped data or recorded empty games

In [20]:
con.execute(f"""
SELECT
  season,
  {PANEL_TEAM_COL} AS team,
  week,
  game_id,
  ST_Load_All_w,
  ST_Load_ScoreLinked_w,
  ST_Load_NonScore_w
FROM team_week_panel
WHERE is_reg_game_week = 1
  AND has_st_data_team_season = 1
  AND ST_Load_All_w = 0
ORDER BY season, team, week
LIMIT 200
""").df()

,season,team,week,game_id,ST_Load_All_w,ST_Load_ScoreLinked_w,ST_Load_NonScore_w


Quick sanity check to confirm that row-level parity is maintained between the primary and also NonScore shock features while ensuring the pointer logic was applied accurately during the persistence phase

In [21]:
con.execute("""
SELECT
  SUM(CASE WHEN ST_Shock_Primary_w != ST_Shock_NonScore_w THEN 1 ELSE 0 END) AS primary_mismatch_rows,
  COUNT(*) AS total_rows
FROM team_week_panel
""").df()

,primary_mismatch_rows,total_rows
0,0.0,6782


Quick sanity check to confirm that the physical schema of the team-week panel has been updated to include the full suite of normalization metrics and also binary shock indicators

In [22]:
final_cols = set(con.execute("DESCRIBE team_week_panel").df()["column_name"].tolist())

need_now = [
    "mean_ST_All",
    "sd_ST_All",
    "mean_ST_ScoreLinked",
    "sd_ST_ScoreLinked",
    "mean_ST_NonScore",
    "sd_ST_NonScore",
    "Z_ST_All_w",
    "Z_ST_ScoreLinked_w",
    "Z_ST_NonScore_w",
    "ST_Shock_All_w",
    "ST_Shock_ScoreLinked_w",
    "ST_Shock_NonScore_w",
    "ST_Shock_Primary_w",
    "Z_ST_Primary_w",
]

missing_now = [c for c in need_now if c not in final_cols]

print("Missing columns", missing_now)
print("OK" if not missing_now else "STOP, Step 5 columns missing from stored panel")

Missing columns []
OK


Quick sanity check to confirm that the table schema matches our intended blueprint exactly and also that any "garbage" columns from the join process were dropped before the final write

In [23]:
con.execute("""
SELECT column_name
FROM duckdb_columns
WHERE table_name = 'team_week_panel'
  AND (column_name LIKE '%sched%' OR column_name LIKE 'is_reg%' OR column_name LIKE 'has_st%')
ORDER BY column_name
""").df()

,column_name
0,has_st_data_team_season
1,is_reg_game_week
2,sched_game_type
